# Hashtable Collision Resolution

When we learned about hash tables, we didn't cover what to do with hash collisions. That is covered in this lesson.

A quick recap:
- Hash tables map a **key** to a **value**
- In the implementation, a hash table (typically) stores values in an array
  - The array index at which the key-value pair is stored is determined by a **hash function**
  - A hash function always returns the same index for the same key
- If the hash function returns the same array index for two different keys, this is known as a **hash collision**
- There are multiple ways to resolve hash collisions

Here, we will learn one such way, known as **linear probing**.

## Linear Probing

Most well-written hash tables will rarely encounter hash collisions. That actually makes things difficult for us; we want to see what happens when a hash collision occurs!

Let's start with a bad hash function that easily causes hash collisions:

In [1]:
def hash(string):
    '''
    Returns the sum of ASCII codes of the string.
    This is a lousy hash function!
    '''
    total = 0
    for char in string:
        total += ord(char)
    return total

We are going to look at two `HashTable` implementations.

I will write a naive (i.e. dumb) one that does not resolve collisions, and then you will write one that resolves collisions with linear probing.

### `HashTable` base class

For convenience, let's make both implementations inherit from a base class. We will also make all attributes public for convenience, although this is not recommended for working code.

Each `HashTable` class needs `get(key)`, `set(key, value)`, and `delete(key)` methods to retrieve values, add/change values, and delete values respectively.

In [2]:
class BaseHashTable:
    '''
    Base class for HashTable implementations
    '''
    def __init__(self, size=1000):
        self.size = size
        self.data =  [None] * self.size

    def __repr__(self):
        return f'HashTable(size={self.size})'

    def hash_index(self, key):
        '''
        Takes in a key and returns a hash index (integer that is
        guaranteed not to raise IndexError).
        The hash() function is implemented outside the class so that
        it can be replaced by other hashing functions which also
        return numbers from keys.
        '''
        num = hash(key)
        return num % self.size

    def get(self, key):
        # NOT IN EXAM SYLLABUS
        # This is Python's way of reminding you that a particular method
        # is not implemented by the parent class, and needs to be
        # implemented by a child class.
        # Attempting to call this method when it is not implemented
        # leads to Python raising a `NotImplementedError`
        raise NotImplementedError

    def set(self, key, value):
        raise NotImplementedError

    def delete(self, key):
        raise NotImplementedError
    
    def array(self, start=None, end=None):
        '''
        Displays content of internal array.
        Note: A typical hashtable should not expose its underlying array!
        '''
        for i, tup in enumerate(self.data):
            if start and i < start:
                continue
            if end and i > end:
                continue
            if tup is None:
                print(f"[{i}] <-- 'None'")
            else:
                key, value = tup
                print(f"[{i}] <-- '{key}': '{value}'")

Let's write a naive `HashTable` that does not carry out collision resolution:

In [3]:
class NaiveHashTable(BaseHashTable):
    '''
    This HashTable stores the key and value at the given array index.
    '''
    def get(self, key):
        index = self.hash_index(key)
        s_key, value = self.data[index]
        if key == s_key:  # check if given key matches stored key
            return value
        else:
            return None  # No matching key found

    def set(self, key, value):
        index = self.hash_index(key)
        self.data[index] = (key, value)

    def delete(self, key):
        index = self.hash_index(key)
        del self.data[index]

Let's store some data and then try to retrive it:

In [4]:
# This cell for testing your code

ht = NaiveHashTable()
ht.set('a9', 'hi')
ht.set('b8', 'bye')

# This line should return 'hi'
print(ht.get('a9'))

# This line should return 'bye'
print(ht.get('b8'))

None
bye


That doesn't look right ... time to debug.

In [5]:
# Let's inspect the contents of ht:

ht.array()

[0] <-- 'None'
[1] <-- 'None'
[2] <-- 'None'
[3] <-- 'None'
[4] <-- 'None'
[5] <-- 'None'
[6] <-- 'None'
[7] <-- 'None'
[8] <-- 'None'
[9] <-- 'None'
[10] <-- 'None'
[11] <-- 'None'
[12] <-- 'None'
[13] <-- 'None'
[14] <-- 'None'
[15] <-- 'None'
[16] <-- 'None'
[17] <-- 'None'
[18] <-- 'None'
[19] <-- 'None'
[20] <-- 'None'
[21] <-- 'None'
[22] <-- 'None'
[23] <-- 'None'
[24] <-- 'None'
[25] <-- 'None'
[26] <-- 'None'
[27] <-- 'None'
[28] <-- 'None'
[29] <-- 'None'
[30] <-- 'None'
[31] <-- 'None'
[32] <-- 'None'
[33] <-- 'None'
[34] <-- 'None'
[35] <-- 'None'
[36] <-- 'None'
[37] <-- 'None'
[38] <-- 'None'
[39] <-- 'None'
[40] <-- 'None'
[41] <-- 'None'
[42] <-- 'None'
[43] <-- 'None'
[44] <-- 'None'
[45] <-- 'None'
[46] <-- 'None'
[47] <-- 'None'
[48] <-- 'None'
[49] <-- 'None'
[50] <-- 'None'
[51] <-- 'None'
[52] <-- 'None'
[53] <-- 'None'
[54] <-- 'None'
[55] <-- 'None'
[56] <-- 'None'
[57] <-- 'None'
[58] <-- 'None'
[59] <-- 'None'
[60] <-- 'None'
[61] <-- 'None'
[62] <-- 'None'
[6

Oops, the `'b8': 'bye'` key-value pair has overwritten the `'a9': 'hi'` pair!

That is because both `'a9'` and `'b8'` will give the same array index:

In [6]:
print('Array index from a9:', hash('a9'))
print('Array index from b8:', hash('b8'))

# This is a rare occurrence with a good hash function,
# but quite likely to happen with our crappy hash function

Array index from a9: 154
Array index from b8: 154


Let's prevent `'b8'` from overwriting `'a9'` by implementing collision resolution, in this case using the linear probing method.

## Linear Probing

The [linear probing method](https://en.wikipedia.org/wiki/Linear_probing) is simple:

1. If the array index returned by the hash function already has data stored at that index,
2. increment the index by 1 ...
3. And check if the array cell at that index is empty.
4. If it is empty, store the data there. If not, repeat from step 2.

### Implementation details

When we store `'a9': 'hi'`, our HashTable will check index 154, and see that it is unused. It will store the key and value at index 154 of the array.

When we store `'b8': 'bye'`, our HashTable will check index 154 and see that it already contains key `'a9'`. It will then increment the array index by 1, to give 155. It will check index 155, see that it is unused, then store the key and value at index 155 of the array.

### Your Task

Implement a class, `HashTableWithLinearProbe1`, that inherits from `BaseHashTable`.

`HashTableWithLinearProbe1` should implement linear probing for `get()` and `set()` methods. (`delete()` will be covered in the next section.)

In [7]:
class HashTableWithLinearProbe1(BaseHashTable):
    '''
    This HashTable implements linear probing to avoid key collisions.
    '''
    # Read and understand the code below, then try to implement it yourself
    def get(self, key):
        # Your code here
        index = self.hash_index(key)
        while (self.data[index] is not None):
            s_key, value = self.data[index]
            if key == s_key:  # If matching key is found
                return value
            else:
                index += 1  # Increment array index and check again
        return None  # key not found
    
    def set(self, key, value):
        # Your code here
        index = self.hash_index(key)
        found = False
        while (not found) and (self.data[index] is not None):
            s_key, s_value = self.data[index]
            if key == s_key:  # key already exists
                self.data[index] = (key, value)  # overwrite matching key
                found = True
            else:
                index += 1  # Increment array index and check again            
        self.data[index] = (key, value)  # unused index found
    
    def delete(self, key):
        index = self.hash_index(key)
        deleted = False
        while (not deleted) and (self.data[index] is not None):
            s_key, value = self.data[index]
            if key == s_key:  # If matching key is found
                self.data[index] = None
                deleted = True
            else:
                index += 1  # Increment array index and check again

In [8]:
# Use this cell to test your code

ht = HashTableWithLinearProbe1()
ht.set('a9', 'hi')
ht.set('b8', 'bye')

# This line should return 'hi'
print(ht.get('a9'))

# This line should return 'bye'
print(ht.get('b8'))

hi
bye


## Key deletion

Key deletion is a little trickier. If we delete a key and the next index is occupied, this might cause problems for key retrieval at subsequent array indexes (because linear probing will turn up this index as unused, causing the probe to stop at this index).

If the next index is occupied, we will need to find an existing key-value pair to replace this index.

### Implementation details

We are going to add some keys to the hash table:

In [9]:
ht = HashTableWithLinearProbe1()
ht.set('a9', 'hi')
ht.set('b8', 'bye')
ht.set('c7', 'hello again')
ht.set('d9', 'not moving')
ht.set('e5', 'hello over there')
ht.set('f4', 'another one')
ht.set('g3', 'yet another')

print("After initialisation:")
ht.array(154, 161)

After initialisation:
[154] <-- 'a9': 'hi'
[155] <-- 'b8': 'bye'
[156] <-- 'c7': 'hello again'
[157] <-- 'd9': 'not moving'
[158] <-- 'e5': 'hello over there'
[159] <-- 'f4': 'another one'
[160] <-- 'g3': 'yet another'
[161] <-- 'None'


The array index where each key-value pair is stored (let's call this the storage index) does not match the calculated index (returned by `HashTableWithLinearProbe1.hash_index()`):

In [10]:
for key in ('a9', 'b8', 'c7', 'd9', 'e5'):
    print(f"'{key}': calculated index is {hash(key)}")

'a9': calculated index is 154
'b8': calculated index is 154
'c7': calculated index is 154
'd9': calculated index is 157
'e5': calculated index is 154


If we naively try to delete some keys, setting their respective storage indexes to zero, it will break key retrieval for some keys:

In [11]:
print("\nAfter deleting key 'c7':")

ht.delete('c7')
ht.array(154, 161)

print("\nRetrieving key 'd9': should give 'not moving'")
print('>', ht.get('d9'))

print("\nRetrieving key 'e5': should give 'hello over there'")
print('>', ht.get('e5'))


After deleting key 'c7':
[154] <-- 'a9': 'hi'
[155] <-- 'b8': 'bye'
[156] <-- 'None'
[157] <-- 'd9': 'not moving'
[158] <-- 'e5': 'hello over there'
[159] <-- 'f4': 'another one'
[160] <-- 'g3': 'yet another'
[161] <-- 'None'

Retrieving key 'd9': should give 'not moving'
> not moving

Retrieving key 'e5': should give 'hello over there'
> None


Linear probing has failed for key `'e5'`. Its calculated index is `154`, which is occupied. Linear probing brings it to index `156` which is `None`; the hashtable thinks key `'e5'` doesn't exist.

We will need to tweak the deletion algorithm to bring forward the subsequent indices.

Note that we cannot move `157`: key `'d9'`'s calculated index is `157`, which matches its stored index! If we move it to `156`, `get()` will never find it.

### Non-naive key deletion

Let's override the `delete()` method of our `HashTableWithLinearProbe` by subclassing it:

In [12]:
class HashTableWithLinearProbe2(HashTableWithLinearProbe1):
    def delete(self, key):
        index = self.hash_index(key)
        deleted = False
        while (not deleted) and (self.data[index] is not None):
            s_key, s_value = self.data[index]
            if key == s_key:  # If matching key is found
                self.data[index] = None
                deleted = True
            else:
                index += 1  # Increment array index and check again
        # From this point, we need to continue incrementing the array index
        # If a key-value pair is found at the index, check:
        # - whether it is still retrievable via its key
        # If the retrieved key-value is no longer correct, need to store it
        # in the hashtable again for array index to be recalculated.

        index += 1
        while (self.data[index] is not None):
            s_key, s_value = self.data[index]
            
            # Let's see what index the hashtable will actually retrieve
            c_index = self.hash_index(s_key)  # calculated index
            c_key, c_value = self.data[c_index]
            
            # See if there will be problems retrieving this key_value
            if (s_key, s_value) == (c_key, c_value):
                pass  # no problems retrieving this key-value pair; skip
            else:
                # Have to delete and re-add (s_key, s_value) again
                self.data[index] = None
                self.set(s_key, s_value)
            index += 1

In [13]:
ht = HashTableWithLinearProbe2()
ht.set('a9', 'hi')
ht.set('b8', 'bye')
ht.set('c7', 'hello again')
ht.set('d9', 'not moving')
ht.set('e5', 'hello over there')
ht.set('f4', 'another one')
ht.set('g3', 'yet another')

print("After initialisation:")

ht.array(154, 161)

print("\nAfter deleting key 'c7':")

ht.delete('c7')
ht.array(154, 161)

print("\nRetrieving key 'd9': should give 'not moving'")
print('>', ht.get('d9'))

print("\nRetrieving key 'e5': should give 'hello over there'")
print('>', ht.get('e5'))

After initialisation:
[154] <-- 'a9': 'hi'
[155] <-- 'b8': 'bye'
[156] <-- 'c7': 'hello again'
[157] <-- 'd9': 'not moving'
[158] <-- 'e5': 'hello over there'
[159] <-- 'f4': 'another one'
[160] <-- 'g3': 'yet another'
[161] <-- 'None'

After deleting key 'c7':
[154] <-- 'a9': 'hi'
[155] <-- 'b8': 'bye'
[156] <-- 'e5': 'hello over there'
[157] <-- 'd9': 'not moving'
[158] <-- 'f4': 'another one'
[159] <-- 'g3': 'yet another'
[160] <-- 'None'
[161] <-- 'None'

Retrieving key 'd9': should give 'not moving'
> not moving

Retrieving key 'e5': should give 'hello over there'
> hello over there


Now that we have re-processed all the storage indices after the deleted index, the `HashTable` works properly again.

# Summary

1. Key collision happens when a hash function returns the same array index for two (or more) keys
2. To avoid overwriting existing key-value pairs, the hash table must implement a method of resolving key collisions
3. One simple method for doing so is linear probing:
   - Retrieving data: if data without a matching key already exists at a storage index, increment the storage index and check again until a matching key is found. If an empty index is encountered, the key does not exist.
   - Storing data: if data without a matching key already exists at a storage index, increment the storage index and check again until a matching key is found. If an empty index is encountered, store the key-value at that index.
   - Deleting data: if data without a matching key already exists at a storage index, increment the storage index and check again until a matching key is found. After deletion, check the subsequent storage indexes to see if they can be successfully retrieved. If they cannot, remove and re-add them again.